Author: Nicholas Steele
Snow Survey Hydrologist
USDA - Natural Resources Conservation Service
1201 NE Lloyd Blvd, Suite #900
Portland, OR  97232
Email: nick.steele@usda.gov
Cell: 503-819-5880


# Imports

In [86]:
# from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
# from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn import svm
# from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
# from functools import reduce

import dash
from dash import dcc
from dash import html
# import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
from datetime import date
import SNOTEL_RegressionTool as RegressionTool

import requests
import xmltodict
# import datetime as dt

# import RegressionFun

In [2]:
apple = RegressionTool.RegressionFun([{'302:OR:SNTL':'WTEQ'},  {'302:OR:SNTL':'SNWD'}, {'302:OR:SNTL':'TAVG'}, {'653:OR:SNTL':'WTEQ'}], '01/01/2010','02/01/2020')

In [6]:
apple.train_model('Linear', 0.3)

RMSE for training set 3.570029415020483
RMSE for test set 8.112468786361747


In [7]:
apple.train_test_fig

In [5]:
apple.model_fit_fig

In [6]:
apple.make_predictions('03/01/2020', '04/01/2020')

In [7]:
apple.predictions_plot

# Dash Web App

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# cd /content/gdrive/My Drive/Projects/Work/SNOTELRegressionTool

In [ ]:
Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')
Stations

In [ ]:
stations = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

# stations_complete = dict(zip(stations, triplets))

# labels = {'label': stations}
# values = {'value':triplets}

In [ ]:
options = []
for i in range(len(Stations.index)):
  options.append({'label': stations[i], 'value': triplets[i]})

In [ ]:
options

In [ ]:
# import itertools

# def zip_with_scalar(l, o):
#     return zip(l, itertools.repeat(o))

In [ ]:
# key = 'label'
# labels = list(map(lambda i: (key, i), Sites))
# labels

In [130]:
str(date(2017, 8, 25))

'2017-08-25'

In [ ]:
!dir

In [ ]:
Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')

In [ ]:

#----------------------------------------------------------------------
# variables to be used in different places within app

heading={'color':'black', 'font-size':50,'text-Align':'center','font-weight': 'bold'}
subheading={'color':'black', 'font-weight': 'bold', 'text-Align':'left'}
station_dropdown_style = {'width': '45%', 'display': 'inline-block', 'float':'left'}
parameter_dropdown_style = {'width': '45%', 'display': 'inline-block', 'position': 'relative', 'float':'right'}

Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')
station_names = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

options = []
for i in range(len(Stations.index)):
  options.append({'label': station_names[i], 'value': triplets[i]})
           
parameter_options =  [{'label': 'Snow Water Equivalent', 'value': 'WTEQ'},
                      {'label': 'Accumulative Precipitation', 'value': 'PREC'},
                      {'label': 'Precipitation', 'value': 'PRCP'},
                      {'label': 'Snow Depth', 'value': 'SNWD'},
                      {'label': 'Average Temperature', 'value': 'TAVG'},
                      {'label': 'Observed Temperature', 'value': 'TOBS'},
                      {'label': 'Max Temperature', 'value': 'TMAX'},
                      {'label': 'Min Temperature', 'value': 'TMIN'}]

model_options = [{'label':'Linear Regression', 'value': 'Linear'},
                {'label':'Ridge Regression', 'value': 'Ridge'},
                {'label':'Lasso Regression', 'value': 'Lasso'},
                {'label':'Huber Regression', 'value': 'Huber'},
                {'label':'Support Vector Machines', 'value': 'SVM'},
                {'label':'Random Forest Regression', 'value': 'Random Forest'},
                {'label':'AdaBoost Regression', 'value': 'AdaBoost'},
                {'label':'GradientBoost Regression', 'value': 'GradientBoost'}]

#----------------------------------------------------------------------

# Initialize the app class

app = dash.Dash(__name__)
#https://stackoverflow.com/questions/50844844/python-dash-custom-css
#----------------------------------------------------------------------

# app layout

app.layout = html.Div(children = [
    
    html.Div(children = [
        html.Br(),
        html.Label(['Select the station and parameter to be used as the response:'], style=subheading),
        dcc.Dropdown(id = 'response-station', 
                     options = options, 
                     multi=False,
                     value = '301:CA:SNTL',
#                      style = station_dropdown_style
                    ),
        dcc.Dropdown(id = 'response-parameter', 
                    options = parameter_options,
                    multi=False,
                    value = "WTEQ",
#                     style = parameter_dropdown_style
                    )]),    
    
    html.Div(children = [
        html.Label(['Select station parameter pairs to be used as predictors:'], style=subheading),
        html.Br(),
        dcc.Dropdown(id = 'predictor-station1', 
                     options = options, 
                     multi = False,
                     value = '391:CA:SNTL',
#                      style = station_dropdown_style,
                    ),
        dcc.Dropdown(id = 'predictor-parameter1', 
                    options = parameter_options,
                    multi = False,
                    value="WTEQ",
#                     style = parameter_dropdown_style,
                    ),
                       
#         html.Br(),
#         dcc.Dropdown(id = 'predictor-station2', 
#                      options = options, 
#                      multi = False,
#                      value = 'predictor',
# #                      style = station_dropdown_style
#                     ),
#         dcc.Dropdown(id = 'predictor-parameter2', 
#                     options = parameter_options,
#                     multi = False,
#                     value = "parameter",
# #                     style = parameter_dropdown_style,
#                     ),
    
                     
#         html.Br(),
#         dcc.Dropdown(id = 'predictor-station3', 
#                      options = options, 
#                      multi = False,
#                      value = 'predictor',
# #                      style = station_dropdown_style
#                     ),
#         dcc.Dropdown(id = 'predictor-parameter3', 
#                     options = parameter_options,
#                     multi = False,
#                     value="parameter",
# #                     style = 'station_dropdown_style'
#                     ),
    ]),
    html.Br(),
    html.Div(children = [

    html.Br(),
    html.Label(['Select data range to train the regression model: '], style=subheading),
    html.Br(),
        

    dcc.DatePickerSingle(
        id='startdate_picker',
#         min_date_allowed=date(1995, 8, 5),
#         max_date_allowed=date(2017, 9, 19),
#         initial_visible_month=date(2017, 8, 5),
        date = date(2020,11,1)
#         date = '09/01/2021'
    ),
    dcc.DatePickerSingle(
        id='enddate_picker',
#         min_date_allowed=date(1995, 8, 5),
#         max_date_allowed=date(2017, 9, 19),
#         initial_visible_month=date(2017, 8, 5),
        date = date(2021,5,1) #'09/20/2021',
    ),
    html.Br(),
    html.Label(['Select Regression Model to be used:']),
    dcc.Dropdown(id = 'model_selection',
                options = model_options,
                value = 'Linear'
    ),    
#     dcc.DatePickerRange(
#         id='my-date-picker-range',  # ID to be used for callback
#         calendar_orientation='horizontal',  # vertical or horizontal
#         day_size=39,  # size of calendar image. Default is 39
#         end_date_placeholder_text="Return",  # text that appears when no end date chosen
#         with_portal=False,  # if True calendar will open in a full screen overlay portal
#         first_day_of_week=0,  # Display of calendar when open (0 = Sunday)
#         reopen_calendar_on_clear=True,
#         is_RTL=False,  # True or False for direction of calendar
#         clearable=True,  # whether or not the user can clear the dropdown
#         number_of_months_shown=1,  # number of months shown when calendar is open
#         min_date_allowed=date(2018, 1, 1),  # minimum date allowed on the DatePickerRange component
#         max_date_allowed=date(2020, 6, 20),  # maximum date allowed on the DatePickerRange component
#         initial_visible_month=date(2020, 5, 1),  # the month initially presented when the user opens the calendar
#         start_date=date(2018, 8, 7),
#         end_date=date(2020, 5, 15),
#         display_format='MMM Do, YY',  # how selected dates are displayed in the DatePickerRange component.
#         month_format='MMMM, YYYY',  # how calendar headers are displayed when the calendar is opened.
#         minimum_nights=2,  # minimum number of days between start and end date

#         persistence=True,
#         persisted_props=['start_date'],
#         persistence_type='session',  # session, local, or memory. Default is 'local'

#         updatemode='singledate'  # singledate or bothdates. Determines when callback is triggered
#     ),
        
    html.Br(),
    html.Button(id='submit-button-state', 
#                 value={},
               children = 'Submit',
#                    style={'fontsize:24'}
        )
    ]),
    

    html.Div(children = [
            dcc.Graph(id='training vs test plot', figure={},style={'width': '80%','padding-left':'2px','float':'right'}),
            dcc.Graph(id='regression model plot', figure={},style={'width': '80%','padding-left':'2px','float':'right'}),
            html.Br(),
            html.Pre(id='test space', children=[]),
#             style={'white-space': 'pre-wrap','word-break': 'break-all',
#                  'border': '1px solid black','text-align': 'left',
#                  'padding': '12px 12px 12px 12px', 'color':'blue',
#                  'margin-top': '3px'})    
        ]),
    html.Div(children = [
            dcc.DatePickerSingle(id='predict_startdate_picker',
            date = None
            ),
            dcc.DatePickerSingle(id='predict_enddate_picker',
            date = None
            ),
#             dcc.Graph(id='predictions plot', figure={},style={'width': '80%','padding-left':'2px','float':'right'}
#             )
                                
        
        
    ]),

])

#-----------------callback for regression plots - using INPUT ------------------------

# @app.callback(
#     [Output('training vs test plot', 'figure'),
#     Output('regression model plot','figure')],
#     [Input('response-station','value'),
#     Input('response-parameter','value'),
#     Input('predictor-station1','value'),
#     Input('predictor-parameter1','value'),
#     #                State('predictorstation2','predictor'),               
#     #                State('predictorparameter2','parameter'),
#     #                State('predictorstation3','predictor'),
#     #                State('predictorparameter3','parameter'),
#     Input('startdate_picker', 'date'),
#     Input('enddate_picker', 'date'),
#     Input('model_selection', 'value')
#     ])

# def plots(
#     responsestation, 
#     responseparameter, 
#     predictorstation1, 
#     predictorparameter1, 
# #     predictorstation2, 
# #     predictorparameter2, 
# #     predictorstation3, 
# #     predictorparameter3
# #     ):
#     startdate, 
#     enddate,
#     modelselection
#     ):
# #     responsestation = '301:CA:SNTL'
# #     responseparameter = 'WTEQ'
# #     predictorstation1 = '391:CA:SNTL'
# #     predictorparameter1 = 'WTEQ' 

# #     if start_date is not None:
# #         start_date = date.fromisoformat(start_date)
# # #         start_date_string = start_date_object.strftime('%B %d, %Y')
# #     if end_date is not None:
# #         end_date = date.fromisoformat(end_date)
    
#     model = RegressionTool.RegressionFun([
#         {responsestation:responseparameter}, 
#         {predictorstation1:predictorparameter1}, 
# #         {predictorstation2:predictorparameter2}, 
# #         {predictorstation3:predictorparameter3}
#         ], 
#         str(startdate), str(enddate))
    
#     model.train_model(modelselection, 0.3)
    

#     return model.train_test_fig, model.model_fit_fig  

#-----------------callback for regression plots - using STATE ------------------------

@app.callback(
    [Output('training vs test plot', 'figure'),
    Output('regression model plot','figure')],
    [Input('submit-button-state', component_property='n_clicks')
    ],
    [State('response-station','value'),
    State('response-parameter','value'),
    State('predictor-station1','value'),
    State('predictor-parameter1','value'),
    State('startdate_picker', 'date'),
    State('enddate_picker', 'date'),
    State('model_selection', 'value')
    ])

# @app.callback(
#     [Output('training vs test plot', 'figure'),
#     Output('regression model plot','figure')],
# #     [Input('submit-button-state', component_property='value'),
# #     Input('submit-button-state', component_property='date'),
#     [Input('response-station','value'),
#     Input('response-parameter','value'),
#     Input('predictor-station1','value'),
#     Input('predictor-parameter1','value'),
#     Input('startdate_picker', 'date'),
#     Input('enddate_picker', 'date'),
#     Input('model_selection', 'value')
#     ])

def plots(n,
    responsestation, 
    responseparameter, 
    predictorstation1, 
    predictorparameter1, 
#     predictorstation2, 
#     predictorparameter2, 
#     predictorstation3, 
#     predictorparameter3
#     ):
    startdate, 
    enddate,
    modelselection
    ):
#     responsestation = '301:CA:SNTL'
#     responseparameter = 'WTEQ'
#     predictorstation1 = '391:CA:SNTL'
#     predictorparameter1 = 'WTEQ' 

#     if start_date is not None:
#         start_date = date.fromisoformat(start_date)
# #         start_date_string = start_date_object.strftime('%B %d, %Y')
#     if end_date is not None:
#         end_date = date.fromisoformat(end_date)
    print(n)
    model = RegressionTool.RegressionFun([
        {responsestation:responseparameter}, 
        {predictorstation1:predictorparameter1}, 
#         {predictorstation2:predictorparameter2}, 
#         {predictorstation3:predictorparameter3}
        ], 
        str(startdate), str(enddate))
    
    model.train_model(modelselection, 0.3)
    

    return model.train_test_fig, model.model_fit_fig  





#----------------Predictions plot callback-----------------
# @app.callback(
#     [Output('predictions plot', 'figure')],
#     [State('predict_startdate_picker', 'date'),
#     State('predict_enddate_picker', 'date'),
#     ])

# def predictions(predict_startdate, predict_enddate):
#     predict_startdate = '11/01/2020'
#     predict_enddate = '3/01/2021'
#     model.make_predictions(predict_startdate, predict_enddate)
#     return model.predictions_plot






#----------------Testing callbacks------------------------

# @app.callback(Output('test space', 'children'),
#               [Input('response-station','value'),
#               Input('response-parameter', 'value'),
#               Input('predictor-station1','value'),
#               Input('predictor-parameter1','value'),
# #               Input('startdate-picker', 'date'),
# #               Input('enddate-picker', 'date')
#               ])

# def display_value(value, other_value, par, bla):
# #     start_date_object = date.fromisoformat(dateroo)
# #     start_date_string = start_date_object.strftime('%B %d, %Y')
#     return value, other_value, par, bla
    

    
    
# @app.callback(
#     Output('test space', 'children'),
#     [Input('startdate_picker', 'date'),
#      Input('enddate_picker', 'date')]
#     )    
    
# def display_date(start_date, end_date):
#     a = str(start_date)
#     return a, end_date
       
    
# @app.callback(
#     Output('test space', 'children'),
#     [Input('my-date-picker-range', 'start_date'),
#      Input('my-date-picker-range', 'end_date')]
#     )
    
# def display_dates(start_date, end_date):
# #     print(str(start_date))
# #     print(str(end_date))
# #     a = str(start_date)
# #     b =str(end_date)
# #     return a,b
#     return str(start_date), str(end_date)
    
#----------------------------------------------------------------------------------------------------------
#Run the app

if __name__ == '__main__':
              app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 12:48:37] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


None


127.0.0.1 - - [12/Oct/2021 12:48:39] "POST /_dash-update-component HTTP/1.1" 200 -


RMSE for training set 0.05785157556450477
RMSE for test set 10.673341891580575
1


127.0.0.1 - - [12/Oct/2021 12:48:46] "POST /_dash-update-component HTTP/1.1" 200 -


RMSE for training set 4.465895443016063
RMSE for test set 69.1641739586398


In [50]:
'a' + 'b'

'ab'

In [69]:
def plots(
    responsestation, 
    responseparameter, 
    predictorstation1, 
    predictorparameter1, 
#     predictorstation2, 
#     predictorparameter2, 
#     predictorstation3, 
#     predictorparameter3
#     ):
    
    start_date, end_date
    ):
    
    model = RegressionTool.RegressionFun([
        {responsestation:responseparameter}, 
        {predictorstation1:predictorparameter1}, 
#         {predictorstation2:predictorparameter2}, 
#         {predictorstation3:predictorparameter3}
        ], 
        start_date, end_date)
    
    model.train_model('SVM', 0.3)
    return model.train_test_fig.show()  

In [77]:
apple = RegressionTool.RegressionFun([{'302:OR:SNTL':'WTEQ'},  {'302:OR:SNTL':'SNWD'}, {'653:OR:SNTL':'WTEQ'}, {'302:OR:SNTL':'TAVG'}, {'653:OR:SNTL':'WTEQ'}], '01/01/2015','02/01/2020') #

In [11]:
apple = RegressionTool.RegressionFun([{'302:OR:SNTL':'WTEQ'},  {'302:OR:SNTL':'SNWD'}], '01/01/2015','02/01/2020') #

In [78]:
apple.train_model('SVM', 0.3)

RMSE for training set 2.095448757576578
RMSE for test set 4.218499632935435


In [79]:
apple.train_test_fig

In [80]:
apple.model_fit_fig

In [81]:
apple.make_predictions('02/01/2020', '03/01/2020')